# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.torch_imports import *
from fastai.io import *

import torch.nn as nn

In [2]:
df_raw = pd.read_csv('../input/train.csv')
df_trn, y_trn, nas = proc_df(df_raw, 'label')

In [3]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 10000
n_trn = len(df_trn)-n_valid

In [4]:
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((32000, 784), (10000, 784), (32000,), (10000,))

In [5]:
X_train, X_valid = X_train.values, X_valid.values

In [6]:
mean = X_train.mean()
std = X_train.std()

X_train = (X_train-mean) / std
X_valid = (X_valid-mean) / std

In [7]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

In [8]:
md = ImageClassifierData.from_arrays('tmp/', (X_train, y_train), (X_valid, y_valid))

In [9]:
loss = nn.NLLLoss()
metrics = [accuracy]
opt = optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In [10]:
set_lrs(opt, 1e-2)

In [11]:
fit(net, md, n_epochs=3, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.227226   0.202488   0.9364    
    1      0.137953   0.140963   0.9559                                                                                
    2      0.115916   0.127414   0.9586                                                                                



[array([0.12741]), 0.9586]

In [12]:
preds = predict(net, md.val_dl)

In [13]:
preds = preds.argmax(1)

In [14]:
np.mean(preds == y_valid)

0.9586